## Data output manipulation

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import glob
import re
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from sklearn.decomposition import PCA
from time import time
import math
from sklearn import cross_validation

/home/plexadmin/.virtualenvs/data-science-bowl-2017/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# constants

data = '/kaggle/dev/data-science-bowl-2017-data/'
stage1 = '/kaggle/dev/data-science-bowl-2017-data/stage1/'
labels = '/kaggle/dev/data-science-bowl-2017-data/stage1_labels.csv'
stage1_processed = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed/'
stage1_features_resnet = '/kaggle/dev/data-science-bowl-2017-data/stage1_features_mx/'
stage1_submission = '/kaggle/dev/data-science-bowl-2017-data/stage1_sample_submission.csv'
naive_submission = '/kaggle/dev/jovan/data-science-bowl-2017/data-science-bowl-2017/submissions/naive_submission.csv'
stage1_processed_pca = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed_pca/'
stage1_features_inception = '/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/cache/'
submissions = '/kaggle/dev/data-science-bowl-2017-data/submissions/'

cifar10_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
cifar_data = "/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/"

In [21]:
ids = list()
for s in glob.glob(stage1_features_inception + "*"):
    id = os.path.basename(s)
    id = re.match(r'inception_cifar10_([a-f0-9].*).pkl' , id).group(1)
    ids.append(id)
ids = pd.DataFrame(ids,  columns=["id"])

df = pd.read_csv(labels)
df = pd.merge(df, ids, how='inner', on=['id'])


x = np.array([np.load(stage1_features_inception + "inception_cifar10_" + s + ".pkl") for s in df['id'].tolist()])

y = df['cancer'].as_matrix()
trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                test_size=0.20)

In [22]:
 # flattening trn_x
for i in range(0, len(trn_x)):
    print('original: ' +  str(trn_x[i].shape))
    trn_x[i] = trn_x[i].flatten()
    print('after: ' +  str(trn_x[i].shape))

# flattening val_x
for i in range(0, len(val_x)):
    print('original: ' +  str(val_x[i].shape))
    val_x[i] = val_x[i].flatten()
    print('after: ' +  str(val_x[i].shape))

original: (268, 2048)
after: (548864,)
original: (350, 2048)
after: (716800,)
original: (318, 2048)
after: (651264,)
original: (305, 2048)
after: (624640,)
original: (332, 2048)
after: (679936,)
original: (352, 2048)
after: (720896,)
original: (262, 2048)
after: (536576,)
original: (270, 2048)
after: (552960,)
original: (346, 2048)
after: (708608,)
original: (369, 2048)
after: (755712,)
original: (268, 2048)
after: (548864,)
original: (302, 2048)
after: (618496,)
original: (331, 2048)
after: (677888,)
original: (338, 2048)
after: (692224,)
original: (302, 2048)
after: (618496,)
original: (294, 2048)
after: (602112,)
original: (332, 2048)
after: (679936,)
original: (332, 2048)
after: (679936,)
original: (342, 2048)
after: (700416,)
original: (292, 2048)
after: (598016,)
original: (365, 2048)
after: (747520,)
original: (325, 2048)
after: (665600,)
original: (312, 2048)
after: (638976,)
original: (380, 2048)
after: (778240,)
original: (346, 2048)
after: (708608,)
original: (374, 2048)
aft

In [23]:
transfer_values_test = val_x
transfer_values_train = trn_x
labels_test = val_y
labels_train = trn_y
# cls_test
# cls_train

print("transfer_values_test : " + str(transfer_values_test.shape))
print("transfer_values_train : " + str(transfer_values_train.shape))
print("labels_test : " + str(labels_test.shape))
print("labels_train : " + str(labels_train.shape))
# print("cls_test : " + str(cls_test.shape))
# print("cls_train : " + str(cls_train.shape))

transfer_values_test : (280,)
transfer_values_train : (1117,)
labels_test : (280,)
labels_train : (1117,)


In [ ]:
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, dataset * image_size))
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [17]:
cls_test = (np.arrange(2) == val_x )